**THIS FILE CONTAINS THE CODE FOR PREDICTION ON THE TEST DATASET USING THE LGBM REGRESSOR**

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
file_1=open('/content/drive/MyDrive/Project Energy Consumption/df_te_red_final_modified.txt','rb')
df_tr_red_final=pickle.load(file_1)

**REMOVING THE FEATURES FROM THE TEST SET WHICH WERE NOT USED FOR MODELLING**

In [ ]:
df_tr_red_final.drop(['cloud_coverage','sea_level_pressure','wind_direction','wind_speed',
                      'is_summer_month','is_pub_holiday'],axis=1,inplace=True)

In [ ]:
df_tr_red_final.drop('row_id',axis=1,inplace=True)

In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/lgbm_model_2.txt','rb')
lgbm_model=pickle.load(file_2)

**HERE WE ARE PREDICTING ON THE TEST SET AND THEN USING EXPM1 TO GET THE ORIGINAL METER READINGS FOR THE FINAL PREDICTIONS**

In [ ]:
y_test=lgbm_model.predict(df_tr_red_final)

In [ ]:
y_test=np.expm1(y_test)

In [ ]:
y_test_round=np.round(y_test,4)

In [ ]:
y_test_df=pd.DataFrame(y_test_round)

In [ ]:
y_test_df.rename(columns={0:'meter_reading'},inplace=True)

In [ ]:
y_test_df['row_id']=y_test_df.index

In [ ]:
y_test_df=y_test_df[['row_id','meter_reading']]

In [ ]:
y_test_df['row_id']=y_test_df['row_id'].astype(int)

**ALL THE  METER READINGS WHICH ARE PREDICTED TO BE LESS THAN ZERO ARE SET TO ZERO AS METER READINGS CANNOT BE ZERO.**

In [ ]:
for i in tqdm(y_test_df[y_test_df['meter_reading']<0].index):
  y_test_df['meter_reading'][i]=0

  0%|          | 0/236930 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 236930/236930 [00:20<00:00, 11445.61it/s]


**CONVERTING THE TEST FILE INTO CSV FORMAT FOR SUBMISSION**

In [ ]:
y_test_df.to_csv('lgbm_2.csv',index=False,header=True)

**FINAL PREDICTED METER READINGS BY THE LGBM MODEL**

In [ ]:
y_test_df.head(10)

,row_id,meter_reading
0,0,135.4708
1,1,67.2099
2,2,12.3847
3,3,268.1527
4,4,839.2800
5,5,20.2121
6,6,101.9343
7,7,425.6315
8,8,207.3551
9,9,202.6072
